In [1]:
from itertools import product
from math import sqrt

import gurobipy as gp
from gurobipy import GRB
import random

# Parameters
with open("D:\\base\\jupyter\\gurobi\\example\\C1.1.txt" ) as f:
    txt = f.readlines()
num_customers = 100
num_facilities = 50
cartesian_prod = list(product(range(num_facilities), range(num_customers)))

costs = list(i.split(" ") for i in txt[2:])
costs = [list(map(eval,cost[:-1])) for cost in costs]
#固定成本
fixed_cost = [cost[1] for cost in costs ]
#运输成本
customs_to_facility = [cost[2:] for cost in costs]
shipping_cost = {(i,j): customs_to_facility[i][j] for i,j in cartesian_prod}    

#需求
demand = [random.randint(50,300) for j in range(num_customers)]
#服务能力
service = [random.randint(50,1000) for i in range(num_facilities)]

k=0

In [2]:
# MIP  model formulation

m = gp.Model('facility_location')

y = m.addVars(num_facilities, vtype=GRB.BINARY, name='Select')
x = m.addVars(cartesian_prod, ub=1, vtype=GRB.CONTINUOUS, name='Assign')

m.addConstrs(gp.quicksum(demand[j]*x[i,j] for j in range(num_customers)) <= y[i]*service[i] for i in range(num_facilities))
m.addConstrs(gp.quicksum(x[(i,j)] for i in range(num_facilities)) == 1 for j in range(num_customers))

m.setObjective(y.prod(fixed_cost) , GRB.MINIMIZE)

m.optimize()


Using license file D:\programs\special\GUROBI\gurobi.lic
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 150 rows, 5050 columns and 10050 nonzeros
Model fingerprint: 0x71c55ef9
Variable types: 5000 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 68340.000000
Presolve removed 12 rows and 600 columns
Presolve time: 0.01s
Presolved: 138 rows, 4450 columns, 8850 nonzeros
Variable types: 4400 continuous, 50 integer (50 binary)

Root relaxation: objective 3.269432e+04, 305 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 32694.3243    0    1 68340.0000 32694.3243  52.2% 

In [3]:
print("objVal:{} \n time:{}".format(m.ObjVal,m.getAttr(GRB.Attr.Runtime)))

for i in range(num_facilities):
    if y[i].X==1:
        print("facilities_open:{}".format(i))
for j in range(num_customers):
    select = dict()
    value = list(x[i,j].X for i in range(num_facilities))
    for i in range(len(value)):
        if value[i] != 0:
            select[i] = value[i]
    print("custome_{}:{}".format(j,select))

objVal:32976.0 
 time:0.24301719665527344
facilities_open:0
facilities_open:1
facilities_open:2
facilities_open:3
facilities_open:5
facilities_open:8
facilities_open:14
facilities_open:17
facilities_open:20
facilities_open:22
facilities_open:24
facilities_open:29
facilities_open:31
facilities_open:38
facilities_open:39
facilities_open:40
facilities_open:41
facilities_open:43
facilities_open:44
facilities_open:45
facilities_open:46
facilities_open:48
custome_0:{38: 1.0}
custome_1:{29: 1.0}
custome_2:{43: 1.0}
custome_3:{5: 0.3960674157303372, 20: 0.6039325842696628}
custome_4:{41: 1.0}
custome_5:{0: 0.8581081081081081, 48: 0.14189189189189189}
custome_6:{5: 1.0}
custome_7:{0: 1.0}
custome_8:{5: 0.6332046332046329, 24: 0.36679536679536695}
custome_9:{45: 1.0}
custome_10:{39: 1.0}
custome_11:{39: 1.0}
custome_12:{1: 1.0}
custome_13:{31: 1.0}
custome_14:{45: 1.0}
custome_15:{24: 1.0}
custome_16:{5: 1.0}
custome_17:{0: 1.0}
custome_18:{20: 0.11434977578475342, 31: 0.39461883408071746, 46: 0